## 24-2 Loss와 Metric

- Loss: 학습 모델 시 학습데이터(train data)를 바탕으로 계산되어, 모델의 파라미터 업데이트에 활용되는 함수
- Metric: 모델 학습 종료 후 테스트데이터(test data)ㄹㄹ 바탕으로 계산되어, 학습된 모델의 성능을 평가하는데 활용되는 함수

MNIST 손글씨 분류 모델 평가
- 분류 모델의 Loss함수: Entropy Loss
- 분류 모델의 Metric(학습모델평가): Acurracy

accuracy가 좋은 학습평가 모델이 될 수 없는 이유?

Discrete한 Accuracy는 Continuous한 Cross Entropy에 비해, 파라미터가 학습되어야 할 방향을 정확하게 제시하지 못한다. 비록 위 예시에서 1번 케이스는 Accuracy는 2번 케이스보다 낮았지만, Loss를 낮추는 쪽으로 더 학습이 진행되면 Accuracy도 높이면서 A, B, C간 확률분포의 차이(0.7-0.15)를 더욱 뚜렷하게 해서 결과적으로는 더 명확한 분류 기준을 학습한 모델이 될 여지가 있다.

Accuracy를 기준으로 학습이 진행된다면 2번 케이스는 이미 완벽한 Accuracy에 도달했으므로 더이상 정교한 학습이 이루어질 여지가 없어서(Accuracy가 loss가 된다면 loss=0) A, B, C간 확률분포의 차이(0.5-0.25)가 더 커지도록 학습이 진행되지 않을 것이다.

## 24-3 Confusion Matrix 와 Precision/Recall

분류 모델로 되돌아가 보자
하지만 우리는 분류모델에서 Accuracy가 항상 좋은 Metric이 되지 않을 수 있다
분류 모델 측면에서 모델의 결과가 이진 분류되는 형태일 때, 모델의 예측 결과와 실제 정답셋을 비교하여 아래의 표처럼 4가지 항목으로 표현하는 Confusion Matrix을 떠올려 보자

- True Positive(TP) - 모델이 양성(Positive)을 양성으로 맞혔을 때
- True Negative(TN) - 모델이 음성(Negative)을 음서으로 맞혔을 때
- False Positive(FP) - 모델이 음성(Negative)을 양성(Positive)으로 잘못 예측했을 때
- False Negative(FN) - 모델이 양성(Positive)을 음성(Negative)으로 잘못 예측했을 때

Accuracy 는 전체 표본 중 정확히 분류된 표본의 수   

병원에 내원한 환자가 암인지 아닌지 구분하는 예측모델을 만드는 경우를 생각해봅시다. 100명의 환자에 대한 예측에 대해 다음과 같은 결과를 얻었습니다.

- TP(실제로 암이면서, 암으로 예측한 결과) = 1
- TN(실제로 정상이고, 정상으로 예측한 결과) = 90
- FN(실제로 암이지만, 정상으로 예측한 결과) = 8
- FP(실제로 정상이지만, 암으로 예측한 결과) = 1

정확도: 0.91   
   
정확도는 이 문제의 평가 척도로 적절한가? 잘못 분류된 9명은 평가 척도에서 어떻게 고려되는게 좋을까?

___
Precision 과 Recall
___


## 24-4 Threshold의 변화에 따른 모델 성능

학습시킨 모델의 출력이 일정 이상 기준선(Threshold)를 넘어가면 우리는 양성이라고 분류한다. 우리가 학습시킨 모델은 Recall이 높을 수록 좋은 모델

- 양성확률이 0.5가 넘으면 양성이라고 분류하는 것보다 양성일 확률이 0.3만 넘으면 양성이라고 분류하도록 해보면 어떨까?

- 모델의 파라미터등은 전혀 변한 것이 없는데 모델의 출력값을 해석하는 방식만 다르게 해도 모델의 성능은 변한다

- 모델의 성능이라는 것은 F1 score같은 숫자 하나로만 규정될 수 있는게 아니다.

- Threshold가 달라지는 것을 고려해 전체적인 모델의 성능을 평가하는 방법으로 PR(Precision and Recall) 커브와 ROC(Reciver Operating Characteristic) 커브를 그리는 두 가지 방법이 있다.

In [1]:
from sklearn import datasets

from sklearn.model_selection import train_test_split
import numpy as np

iris = datasets.load_iris()
X = iris.data
y = iris.target

print(X.shape) # 4개의 feature를 가진 150개의 데이터

(150, 4)


### Task의 재구성
___
붗꽃 데이터에 잡음 데이터를 넣고 성능을 일부러 낮춘다

In [2]:
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

print(X.shape)  # 804개의 feature를 가진 150개의 데이터가 되었습니다.

(150, 804)


In [3]:
#- 0, 1 라벨에 속하는 붓꽃 샘플만 사용하도록 제한합니다.
X_train, X_test, y_train, y_test = train_test_split(X[y < 2], y[y < 2],
                                                    test_size=.5,
                                                    random_state=random_state)

print("훈련, 테스트 셋에 사용된 라벨의 종류: {} ".format( set(y_test)))
print("훈련 데이터 shape   :", X_train.shape)
print("테스트 데이터 shape :", X_test.shape)

훈련, 테스트 셋에 사용된 라벨의 종류: {0, 1} 
훈련 데이터 shape   : (50, 804)
테스트 데이터 shape : (50, 804)


### 모델 학습
___
2개의 랍ㄹ로 나뉜 데이터를 갖고, SVM(Support Vector Machine) 으로 모델을 구성하여 학습시킨후 정확도 확인

In [4]:
from sklearn import svm

classifier = svm.SVC(kernel='poly', random_state=random_state)
classifier.fit(X_train, y_train)

classifier.score(X_test,y_test)

0.48

In [5]:
classifier = svm.SVC(kernel='linear', random_state=random_state)
classifier.fit(X_train, y_train)

classifier.score(X_test,y_test)

0.8

In [6]:
classifier.predict(X_test)

array([0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1])

In [7]:
y_test

array([1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0])

In [9]:
y_score = classifier.decision_function(X_test)
print(y_score)

[-0.29512751  0.28798352  0.17635465  0.19056886  0.38391605 -0.30841065
 -0.10084254 -0.23481309  0.18576987 -0.36011033 -0.15726747 -0.25714889
 -0.14979669  0.02063898  0.04509171 -0.17239443  0.07287957 -0.0689103
 -0.13452462 -0.30697712  0.25404241 -0.28916471 -0.52061453  0.25252233
  0.02177777 -0.10980907  0.37468422  0.35303004 -0.6211302  -0.42920064
 -0.14770647  0.00593404 -0.34735296  0.32245409 -0.19439024  0.1288847
 -0.0320947  -0.23008604 -0.10135548 -0.46962186  0.05184235  0.0609688
  0.05632596  0.44769206 -0.38804349  0.24704844  0.16063684  0.0144203
 -0.03136574  0.11179177]


In [11]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = classifier.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)
rpt_result = classification_report(y_test, y_pred)
print(rpt_result)

[[20  4]
 [ 6 20]]
              precision    recall  f1-score   support

           0       0.77      0.83      0.80        24
           1       0.83      0.77      0.80        26

    accuracy                           0.80        50
   macro avg       0.80      0.80      0.80        50
weighted avg       0.80      0.80      0.80        50



In [13]:
y_pred_new_threshold = classifier.decision_function(X_test) > -0.1
conf_mat = confusion_matrix(y_test, y_pred_new_threshold)
print(conf_mat)
rpt_result = classification_report(y_test, y_pred_new_threshold)
print(rpt_result)

[[18  6]
 [ 5 21]]
              precision    recall  f1-score   support

           0       0.78      0.75      0.77        24
           1       0.78      0.81      0.79        26

    accuracy                           0.78        50
   macro avg       0.78      0.78      0.78        50
weighted avg       0.78      0.78      0.78        50



In [14]:
y_pred_new_threshold = classifier.decision_function(X_test)>-0.2
conf_mat = confusion_matrix(y_test,y_pred_new_threshold)
print(conf_mat)
rpt_

[[12 12]
 [ 2 24]]
